In [1]:
import pandas as pd
import numpy as np
import re
import os
from collections import Counter

Collect the list of txt files for each play and save to spPlays

In [24]:
#use this list to exclude certain plays
excludedPlays = []

spDir = 'shakespeare-plays-plus'
spSubDir = [spDir + '/'+ x for x in os.listdir(spDir) if '.' not in x]
spPlays = []

for subDir in spSubDir:
    spPlays = spPlays + [subDir + '/' + x for x in os.listdir(subDir) if 'txt' in x and x not in excludedPlays]


Read all selected plays  
Skip line if a line contains < (non-text)  
Create CounterDict to store a word Counter for each play  
The keys for CounterDict are play names without '.txt'

In [70]:
CounterDict = {}

for spPlay in spPlays:
    playName = spPlay.split('/')[-1].split('.')[0]
    CounterDict[playName] = Counter()
    with open(spPlay, 'r', encoding='utf-16') as playTxt:
        for line in playTxt:
            if '<' in line:
                continue
            if line.split():
                CounterDict[playName].update(Counter(re.findall('\w+', line.lower())))


To do list:
* leave one word out -- we can "cheat" and check this one once we have counts of words with 1, 2, 3...appearances
* leave one play out
* randomly leave out a number of words--simulate multiple times and we should be able to plot emperical bayes estimate and error range

In [71]:
#create a totalCounter for all plays
totalCounter = Counter()
for playCounter in CounterDict.values():
    totalCounter.update(playCounter)

In [75]:
print('distinct words:', len(totalCounter))
print('total words:', sum(totalCounter.values()))

distinct words: 22859
total words: 822249


we have fewer words than CASI example  
probably because we are using only play data